In [1]:
import wandb
import nltk
from nltk.stem.porter import *
from torch.nn import *
from torch.optim import *
import numpy as np
import pandas as pd
import torch,torchvision
import random
from tqdm import *
from torch.utils.data import Dataset,DataLoader
stemmer = PorterStemmer()
PROJECT_NAME = 'kickstarter-NLP-V6'
device = 'cuda'

In [2]:
data = pd.read_csv('./data.csv')

In [3]:
def tokenize(sentence):
    return nltk.word_tokenize(sentence)

In [4]:
tokenize("$100")

['$', '100']

In [5]:
def stem(word):
    return stemmer.stem(word.lower())

In [6]:
stem('organic')

'organ'

In [7]:
def bag_of_words(tokenized_words,words):
    tokenized_words = [stem(w) for w in tokenized_words]
    bag = np.zeros(len(words))
    for idx,w in enumerate(words):
        if w in tokenized_words:
            bag[idx] = 1.0
    return bag

In [8]:
bag_of_words(['hi'],['hi','how','hi'])

array([1., 0., 1.])

In [9]:
data

        Unnamed: 0                                              blurb  \
0                1  Using their own character, users go on educati...   
1                2  MicroFly is a quadcopter packed with WiFi, 6 s...   
2                3  A small indie press, run as a collective for a...   
3                4  Zylor is a new baby cosplayer! Back this kicks...   
4                5  Hatoful Boyfriend meet Skeletons! A comedy Dat...   
...            ...                                                ...   
215508      215509  This new, designer approach to family wall art...   
215509      215510  Im looking to build a urban youth center to he...   
215510      215511  My plan is to create the most realistic and ac...   
215511      215512  Create unique and fantastic gifts: Mash works ...   
215512      215513  Currently running a shop on Zazzle come and wo...   

             state  
0           failed  
1       successful  
2           failed  
3           failed  
4           failed

In [10]:
import wandb
import nltk
from nltk.stem.porter import *
from torch.nn import *
from torch.optim import *
import numpy as np
import pandas as pd
import torch,torchvision
import random
from tqdm import *
from torch.utils.data import Dataset,DataLoader
stemmer = PorterStemmer()
PROJECT_NAME = 'kickstarter-NLP-V6'
device = 'cuda'

In [11]:
data = pd.read_csv('./data.csv')[:2500]

In [12]:
def tokenize(sentence):
    return nltk.word_tokenize(sentence)

In [13]:
tokenize("$100")

['$', '100']

In [14]:
def stem(word):
    return stemmer.stem(word.lower())

In [15]:
stem('organic')

'organ'

In [16]:
def bag_of_words(tokenized_words,words):
    tokenized_words = [stem(w) for w in tokenized_words]
    bag = np.zeros(len(words))
    for idx,w in enumerate(words):
        if w in tokenized_words:
            bag[idx] = 1.0
    return bag

In [17]:
bag_of_words(['hi'],['hi','how','hi'])

array([1., 0., 1.])

In [18]:
X = data['blurb']
y = data['state']
words = []
data = []
labels = {}
labels_r = {}
idx = 0

In [19]:
for label in y:
    if label not in list(labels.keys()):
        idx += 1
        labels[label] = idx
        labels_r[idx] = label

In [20]:
labels

{'failed': 1, 'successful': 2}

In [21]:
labels_r

{1: 'failed', 2: 'successful'}

In [22]:
for X_batch,y_batch in tqdm(zip(X,y)):
    X_batch = tokenize(X_batch)
    new_X = []
    for Xb in X_batch:
        new_X.append(stem(Xb))
    words.extend(new_X)
    data.append([
        new_X,
        np.eye(labels[y_batch],len(labels))[labels[y_batch]-1]
    ])

In [23]:
data[0][0]

['use',
 'their',
 'own',
 'charact',
 ',',
 'user',
 'go',
 'on',
 'educ',
 'quest',
 'around',
 'a',
 'virtual',
 'world',
 'level',
 'up',
 'subject-ori',
 'skill',
 '(',
 'ie',
 'physic',
 ')',
 '.']

In [24]:
data[0][1]

array([1., 0.])

In [25]:
data[0][5]

In [26]:
data[0][1]

array([1., 0.])

In [27]:
data[98][1]

array([1., 0.])

In [28]:
data[56][1]

array([1., 0.])

In [29]:
data[200][1]

array([0., 1.])

In [30]:
data = pd.read_csv('./data.csv')[:2500].sample(frac=1)

In [31]:
def tokenize(sentence):
    return nltk.word_tokenize(sentence)

In [32]:
tokenize("$100")

['$', '100']

In [33]:
def stem(word):
    return stemmer.stem(word.lower())

In [34]:
stem('organic')

'organ'

In [35]:
def bag_of_words(tokenized_words,words):
    tokenized_words = [stem(w) for w in tokenized_words]
    bag = np.zeros(len(words))
    for idx,w in enumerate(words):
        if w in tokenized_words:
            bag[idx] = 1.0
    return bag

In [36]:
bag_of_words(['hi'],['hi','how','hi'])

array([1., 0., 1.])

In [37]:
X = data['blurb']
y = data['state']
words = []
data = []
labels = {}
labels_r = {}
idx = 0

In [38]:
for label in y:
    if label not in list(labels.keys()):
        idx += 1
        labels[label] = idx
        labels_r[idx] = label

In [39]:
labels

{'failed': 1, 'successful': 2}

In [40]:
labels_r

{1: 'failed', 2: 'successful'}

In [41]:
for X_batch,y_batch in tqdm(zip(X,y)):
    X_batch = tokenize(X_batch)
    new_X = []
    for Xb in X_batch:
        new_X.append(stem(Xb))
    words.extend(new_X)
    data.append([
        new_X,
        np.eye(labels[y_batch],len(labels))[labels[y_batch]-1]
    ])

In [42]:
words = sorted(set(words))
np.random.shuffle(data)

In [43]:
X = []
y = []

In [44]:
for sentence,tag in tqdm(data):
    X.append(bag_of_words(sentence,words))
    y.append(tag)

In [45]:
from sklearn.model_selection import *
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.125,shuffle=False)
X_train = torch.from_numpy(np.array(X_train)).to(device).float()
y_train = torch.from_numpy(np.array(y_train)).to(device).float()
X_test = torch.from_numpy(np.array(X_test)).to(device).float()
y_test = torch.from_numpy(np.array(y_test)).to(device).float()

In [46]:
def get_loss(model,X,y,criterion):
    preds = model(X)
    loss = criterion(preds,y)
    return loss.item()

In [47]:
def get_accuracy(model,X,y):
    preds = model(X)
    correct = 0
    total = 0
    for pred,yb in zip(preds,y):
        pred = int(torch.argmax(pred))
        yb = int(torch.argmax(yb))
        if pred == yb:
            correct += 1
        total += 1
    acc = round(correct/total,3)*100
    return acc

In [48]:
class Model(Module):
    def __init__(self):
        super().__init__()
        self.hidden = 64
        self.linear1 = Linear(len(words),self.hidden)
        self.linear2 = Linear(self.hidden,self.hidden)
        self.linear3 = Linear(self.hidden,len(labels))
    
    def forward(self,X):
        preds = self.linear1(X)
        preds = self.linear2(preds)
        preds = self.linear3(preds)
        return preds

In [49]:
model = Model().to(device)
criterion = MSELoss()
optimizer = Adam(model.parameters(),lr=0.001)
epochs = 100
batch_size = 32

In [50]:
wandb.init(project=PROJECT_NAME,name='baseline')
for _ in tqdm(range(epochs)):
    for i in range(0,len(X_train),batch_size):
        X_batch = X_train[i:i+batch_size]
        y_batch = y_train[i:i+batch_size]
        preds = model(X_batch)
        loss = criterion(preds,y_batch)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    model.eval()
    torch.cuda.empty_cache()
    wandb.log({'Loss':(get_loss(model,X_train,y_train,criterion)+get_loss(model,X_batch,y_batch,criterion)/2)})
    torch.cuda.empty_cache()
    wandb.log({'Val Loss':get_loss(model,X_test,y_test,criterion)})
    torch.cuda.empty_cache()
    wandb.log({'Acc':(get_accuracy(model,X_train,y_train)+get_accuracy(model,X_batch,y_batch))/2})
    torch.cuda.empty_cache()
    wandb.log({'Val Acc':get_accuracy(model,X_test,y_test)})
    torch.cuda.empty_cache()
    model.train()
wandb.finish()
torch.cuda.empty_cache()